# Project 3 - The molecule forge

Dit is mijn jupyter notebook over mijn 3e project, in dit project ga ik werken met AI en proberen om nieuwe fictieve moleculen te maken

Tips voor mijzelf
- Numpy arrays voor speed?

### Het maken van een Virtual Environment
Voor coderen in python is het belangrijk om een Virtual Environment te maken, hiermee kun je per project een eigen python install hebben als het ware. 
Er zijn een aantal redenen om een Venv te gebruiken:
 - Scheiden van projecten, bijvoorbeeld bij het ene project heb je numpy versie 1.5 nodig en bij een ander project heb je versie 2 nodig. Doormiddel van Venv's kun je dit dan makkelijk scheiden.
 - Voorkomt vervuiling van je environment, er zijn geen onnodige packages die mee worden gestuurd als het project word geplubliceerd.
 - Een Venv is makkelijk te maken op een ander systeem waar de packages niet zijn geinstalleerd doormiddel van een requirements.txt.

Stukje voor imports 

In [2]:
from rdkit import Chem
import numpy as np

from itertools import chain
import random

Dit is een functie die ik zelf heb gemaakt om de eerst n aantal regels in een bestand te openen.

In [3]:
def read_lines(n):
    lines = []
    with open("guacamol_v1_train.smiles", "r") as file:
        for l in range(n):
            line = file.readline().strip()
            if not line:
                break
            lines.append(line)
    return lines

Stukje code om de eerste x aantal codes op te slaan en te printen naar de console.

In [4]:
molecules_amount = 100
molecules = read_lines(molecules_amount)
print(molecules)

['CCC(C)(C)Br', 'CCCN(CCc1cccc(-c2ccccc2)c1)C(=O)C1OC(C(=O)O)=CC(N)C1NC(C)=O', 'Oc1ccc(C2CC(c3ccccc3)=NN2C(=S)Nc2ccccc2)cc1', 'CC1(C)OCC2OC3(C4OC(C)(C)OC4CO)OC(C)(C)OC3C2O1', 'COC(=O)c1cc(C(=CCCCC(=O)SC)c2cc(Cl)c(OC)c(C(=O)OC)c2)cc(Cl)c1OC', 'Cc1cc(COc2ccc(NC(=O)C3CN(C)CCC3C(=O)NO)cc2)c2ccccc2n1', 'CCOC(=O)c1ccc(O)c(-n2cc3c(c2-c2ccccc2)c(=O)n(C)c(=O)n3C)c1', 'COc1cc(OC)c2c(-c3cccc(-c4ccc(C#N)cc4)c3)cc(=O)oc2c1', 'COc1cc2[nH]c(C(=O)Nc3ccc(F)cc3)cc2c(OC)c1OC', 'COc1cc(F)cc(-c2ccc(C(CC(=O)O)NC(=O)C3CCCN3S(=O)(=O)c3cc(Cl)cc(Cl)c3)cc2)c1', 'COc1cc(-c2cc(OC)c(-n3c(=O)ccc4cc(S(=O)(=O)Nc5cccnn5)ccc43)cc2F)ccc1Cl', 'CCCCC1(C)CC(CO)C(CCCC)(OC)OO1', 'CCCCCCCCCCCCNc1ccc2c3c(cccc13)C(=O)N(CCCN1CCCNCCNCCCNCC1)C2=O', 'COC(=O)C1=C(c2cc3ccccc3o2)CC2CCC1N2C(=O)NCc1cccc2ccccc12', 'Cc1ccccc1C=Cc1cccc(C(F)(F)P(=O)(O)O)c1', 'Nc1cc(C2CCNCC2)cc(NC2CCCC2)n1', 'CCN(C(C)=O)c1ccc(OC)c2nc(NC(=O)C3CCN(C(=O)c4cccc(C(F)(F)F)c4)CC3)sc12', 'COC(=O)C1CCC(C)C(c2ccc(C)cc2)N1C(=O)c1ccc(C=NOCC(O)COCc2ccco2)cc1', 'CCOC(=O)C1

Ook moet ik een functie hebben om te kijken of de gemaakte moleculen daadwerkelijk echt bestaan, hiervoor gebruik ik de functie MolFromSmiles(), deze functie pakt een van mijn smiles moleculen en splitst deze in losse atomen. De functie plakt deze dan na elkaar en probeert deze in mol formaat van Rdkit te zetten, mocht dit niet lukken dan is dit invalid en krijg ik None terug.

In [5]:
def is_valid_smiles(molecule):
    mol = Chem.MolFromSmiles(molecule)
    if mol is not None:
        return 'Valid'
    else:
       return 'Not valid'

In [6]:
random_molecule_num = random.randint(0, len(molecules) - 1)
random_molecule = molecules[random_molecule_num]

print(random_molecule)
print(is_valid_smiles(random_molecule))

wrong_molecule = random_molecule[0:-1]

print(wrong_molecule)
print(is_valid_smiles(wrong_molecule))

Cc1c(C(=O)OCC(=O)N2CCCC2=O)oc2ccccc12
Valid
Cc1c(C(=O)OCC(=O)N2CCCC2=O)oc2ccccc1
Not valid


[23:14:00] SMILES Parse Error: unclosed ring for input: 'Cc1c(C(=O)OCC(=O)N2CCCC2=O)oc2ccccc1'


### Tokenization
Hieronder staat een functie die een array aan moleculen kan omzetten in losse characters om deze erna naar nummers om te zetten zodat mijn modellen deze nummers kunnen lezen en een predictie op maken.

In [7]:
def molecules_to_tokens(molecules):
    tokens = list()
    for molecule in molecules:
        tokens.append(list(molecule))
    return tokens

tokens = molecules_to_tokens(molecules[0:5])
print(tokens)

[['C', 'C', 'C', '(', 'C', ')', '(', 'C', ')', 'B', 'r'], ['C', 'C', 'C', 'N', '(', 'C', 'C', 'c', '1', 'c', 'c', 'c', 'c', '(', '-', 'c', '2', 'c', 'c', 'c', 'c', 'c', '2', ')', 'c', '1', ')', 'C', '(', '=', 'O', ')', 'C', '1', 'O', 'C', '(', 'C', '(', '=', 'O', ')', 'O', ')', '=', 'C', 'C', '(', 'N', ')', 'C', '1', 'N', 'C', '(', 'C', ')', '=', 'O'], ['O', 'c', '1', 'c', 'c', 'c', '(', 'C', '2', 'C', 'C', '(', 'c', '3', 'c', 'c', 'c', 'c', 'c', '3', ')', '=', 'N', 'N', '2', 'C', '(', '=', 'S', ')', 'N', 'c', '2', 'c', 'c', 'c', 'c', 'c', '2', ')', 'c', 'c', '1'], ['C', 'C', '1', '(', 'C', ')', 'O', 'C', 'C', '2', 'O', 'C', '3', '(', 'C', '4', 'O', 'C', '(', 'C', ')', '(', 'C', ')', 'O', 'C', '4', 'C', 'O', ')', 'O', 'C', '(', 'C', ')', '(', 'C', ')', 'O', 'C', '3', 'C', '2', 'O', '1'], ['C', 'O', 'C', '(', '=', 'O', ')', 'c', '1', 'c', 'c', '(', 'C', '(', '=', 'C', 'C', 'C', 'C', 'C', '(', '=', 'O', ')', 'S', 'C', ')', 'c', '2', 'c', 'c', '(', 'C', 'l', ')', 'c', '(', 'O', 'C', ')', 

Hieronder heb ik een functie geschreven die een set aan tokens pakt en hieruit alle unieke characters haalt, deze worden vervlgens omgezet naar een dictionary die er nummers aan koppelt.

In [8]:
def create_vocab(tokens):
    unique_tokens = set(chain.from_iterable(tokens))
    print(f"Unique tokens: {unique_tokens}")

    sorted_tokens = sorted(unique_tokens)
    print(f"Sorted tokens: {sorted_tokens}")

    char_to_index = {}
    for i, token in enumerate(sorted_tokens, start=1):
        char_to_index[token] = i
    print(f"Character to index mapping: {char_to_index}")
    return char_to_index

vocab = create_vocab(tokens)
print(len(vocab))

Unique tokens: {'B', 'S', 'l', '(', 'C', '-', 'N', ')', 'r', '1', '2', '=', 'c', '4', '3', 'O'}
Sorted tokens: ['(', ')', '-', '1', '2', '3', '4', '=', 'B', 'C', 'N', 'O', 'S', 'c', 'l', 'r']
Character to index mapping: {'(': 1, ')': 2, '-': 3, '1': 4, '2': 5, '3': 6, '4': 7, '=': 8, 'B': 9, 'C': 10, 'N': 11, 'O': 12, 'S': 13, 'c': 14, 'l': 15, 'r': 16}
16


Nu dat ik een vocabulary heb gemaakt kan ik de moleculen die ik heb omgezet naar tokens veranderen in cijfers.

In [9]:
numeric_tokens = [[vocab[char] for char in molecule] for molecule in tokens]
print(numeric_tokens)

[[10, 10, 10, 1, 10, 2, 1, 10, 2, 9, 16], [10, 10, 10, 11, 1, 10, 10, 14, 4, 14, 14, 14, 14, 1, 3, 14, 5, 14, 14, 14, 14, 14, 5, 2, 14, 4, 2, 10, 1, 8, 12, 2, 10, 4, 12, 10, 1, 10, 1, 8, 12, 2, 12, 2, 8, 10, 10, 1, 11, 2, 10, 4, 11, 10, 1, 10, 2, 8, 12], [12, 14, 4, 14, 14, 14, 1, 10, 5, 10, 10, 1, 14, 6, 14, 14, 14, 14, 14, 6, 2, 8, 11, 11, 5, 10, 1, 8, 13, 2, 11, 14, 5, 14, 14, 14, 14, 14, 5, 2, 14, 14, 4], [10, 10, 4, 1, 10, 2, 12, 10, 10, 5, 12, 10, 6, 1, 10, 7, 12, 10, 1, 10, 2, 1, 10, 2, 12, 10, 7, 10, 12, 2, 12, 10, 1, 10, 2, 1, 10, 2, 12, 10, 6, 10, 5, 12, 4], [10, 12, 10, 1, 8, 12, 2, 14, 4, 14, 14, 1, 10, 1, 8, 10, 10, 10, 10, 10, 1, 8, 12, 2, 13, 10, 2, 14, 5, 14, 14, 1, 10, 15, 2, 14, 1, 12, 10, 2, 14, 1, 10, 1, 8, 12, 2, 12, 10, 2, 14, 5, 2, 14, 14, 1, 10, 15, 2, 14, 4, 12, 10]]


### Padding en sifting
Padding en spacing is nodig omdat een AI model elke input van de zelfde lengte moet hebben, een molecuul van bijvoorbeeld een lengte van 10 moet ook samen kunnen met een molecuul van bijvoorbeeld een lengte van 40. Ik doe dit door te kijken welk molecuul in de numeric array de langste is en bij de andere nullen toevoegen tot ik aan de zelfde lengte zit. Bij mijn toepassen zit wel nog een fout, als ik op de hele database ga trainen dan kan het zijn dat er grotere moleculen voorkomen en ik dus mijn model moet aanpassen.

Shifting is het toevoegen van een 0 aan het begin van een molecuul bijvoorbeeld een molecuul als C=O (in SMILES) is dan in tokens "C, =, O" en in nummers [3, 2, 4] doormiddel van shiften worden de nummers dan [0, 3, 2] en weet het model dat die de 0 moet gaan voorspellen, dit is hoe een RNN sequentialy kan leren.

Als ik shifting gebruik moet ik ook één bij de lengte toevoegen om rekening te houden met het extra getal.

In [10]:
def find_longest_molecule(molecules):
    longest = 0
    for n in range(len(molecules)):
        if len(molecules[n]) > longest:
            longest = len(molecules[n])
    return longest + 1

longest = find_longest_molecule(numeric_tokens)
print(f"Longest molecule: {longest}")

Longest molecule: 64


Nu is het tijd om de input en de targets voor het model te maken, dit bestaan uit input_tokens die de numeric token zijn maar dan met een extra 0 ervoor, dit is om het model te laten leren wat erna komt. Ook maak ik hier de target voor het model, dit zijn dan de numeric values maar met een extra 0 om de offset te houden en de lengte het zelfde te houden. Een RNN heeft dan ook de zelfde lengte input en output nodig.

Extra, het is ook een oplossing om een < END > en < START > token te maken om zo nog beter aan te geven wanneer het model moet starten en stoppen met de sequence. Dit kan ik dan in de vocabulary erbij zetten.

In [11]:
def shift_tokens(numeric_tokens, shift_num):
    shifted_tokens = []
    for molecule in numeric_tokens:
        molecule = [shift_num] + molecule    
        shifted_tokens.append(molecule)
    return shifted_tokens

def add_end_padding(numeric_tokens, padding_value):
    padded_tokens = []
    for molecule in numeric_tokens:
        molecule = molecule + [padding_value]
        padded_tokens.append(molecule)
    return padded_tokens



input_tokens = shift_tokens(numeric_tokens, 0)
target_tokens = add_end_padding(numeric_tokens, 0)
print(f"Input tokens: {input_tokens}")
print(f"Target tokens: {target_tokens}")

Input tokens: [[0, 10, 10, 10, 1, 10, 2, 1, 10, 2, 9, 16], [0, 10, 10, 10, 11, 1, 10, 10, 14, 4, 14, 14, 14, 14, 1, 3, 14, 5, 14, 14, 14, 14, 14, 5, 2, 14, 4, 2, 10, 1, 8, 12, 2, 10, 4, 12, 10, 1, 10, 1, 8, 12, 2, 12, 2, 8, 10, 10, 1, 11, 2, 10, 4, 11, 10, 1, 10, 2, 8, 12], [0, 12, 14, 4, 14, 14, 14, 1, 10, 5, 10, 10, 1, 14, 6, 14, 14, 14, 14, 14, 6, 2, 8, 11, 11, 5, 10, 1, 8, 13, 2, 11, 14, 5, 14, 14, 14, 14, 14, 5, 2, 14, 14, 4], [0, 10, 10, 4, 1, 10, 2, 12, 10, 10, 5, 12, 10, 6, 1, 10, 7, 12, 10, 1, 10, 2, 1, 10, 2, 12, 10, 7, 10, 12, 2, 12, 10, 1, 10, 2, 1, 10, 2, 12, 10, 6, 10, 5, 12, 4], [0, 10, 12, 10, 1, 8, 12, 2, 14, 4, 14, 14, 1, 10, 1, 8, 10, 10, 10, 10, 10, 1, 8, 12, 2, 13, 10, 2, 14, 5, 14, 14, 1, 10, 15, 2, 14, 1, 12, 10, 2, 14, 1, 10, 1, 8, 12, 2, 12, 10, 2, 14, 5, 2, 14, 14, 1, 10, 15, 2, 14, 4, 12, 10]]
Target tokens: [[10, 10, 10, 1, 10, 2, 1, 10, 2, 9, 16, 0], [10, 10, 10, 11, 1, 10, 10, 14, 4, 14, 14, 14, 14, 1, 3, 14, 5, 14, 14, 14, 14, 14, 5, 2, 14, 4, 2, 10, 1, 8

Nu moet ik alleen nog zorgen dat alle inputs en targets daadwerkelijk de zelfde lengte hebben als het langste molecuul. Dit is heel makkelijk te doen.

In [33]:
padding_num = 0
def pad_tokens(input_tokens):
    for molecule in input_tokens:
        if len(molecule) < longest:
            molecule += [padding_num] * (longest - len(molecule))
    print(f"tokens after padding: {input_tokens}")
    return input_tokens

padded_inputs = pad_tokens(input_tokens)
padded_targets = pad_tokens(target_tokens)

tokens after padding: [[0, 10, 10, 10, 1, 10, 2, 1, 10, 2, 9, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 10, 10, 10, 11, 1, 10, 10, 14, 4, 14, 14, 14, 14, 1, 3, 14, 5, 14, 14, 14, 14, 14, 5, 2, 14, 4, 2, 10, 1, 8, 12, 2, 10, 4, 12, 10, 1, 10, 1, 8, 12, 2, 12, 2, 8, 10, 10, 1, 11, 2, 10, 4, 11, 10, 1, 10, 2, 8, 12, 0, 0, 0, 0], [0, 12, 14, 4, 14, 14, 14, 1, 10, 5, 10, 10, 1, 14, 6, 14, 14, 14, 14, 14, 6, 2, 8, 11, 11, 5, 10, 1, 8, 13, 2, 11, 14, 5, 14, 14, 14, 14, 14, 5, 2, 14, 14, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 10, 10, 4, 1, 10, 2, 12, 10, 10, 5, 12, 10, 6, 1, 10, 7, 12, 10, 1, 10, 2, 1, 10, 2, 12, 10, 7, 10, 12, 2, 12, 10, 1, 10, 2, 1, 10, 2, 12, 10, 6, 10, 5, 12, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 10, 12, 10, 1, 8, 12, 2, 14, 4, 14, 14, 1, 10, 1, 8, 10, 10, 10, 10, 10, 1, 8, 12, 2, 13, 10, 2, 14, 5, 1

Ik ben nu ook overgestapt naar een NP array en ik heb de inputs X genoemd

In [32]:
X = np.array(padded_inputs)
print(X)
y = np.array(target_tokens)
print(y)

[[ 0 10 10 10  1 10  2  1 10  2  9 16  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0 10 10 10 11  1 10 10 14  4 14 14 14 14  1  3 14  5 14 14 14 14 14  5
   2 14  4  2 10  1  8 12  2 10  4 12 10  1 10  1  8 12  2 12  2  8 10 10
   1 11  2 10  4 11 10  1 10  2  8 12  0  0  0  0]
 [ 0 12 14  4 14 14 14  1 10  5 10 10  1 14  6 14 14 14 14 14  6  2  8 11
  11  5 10  1  8 13  2 11 14  5 14 14 14 14 14  5  2 14 14  4  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0 10 10  4  1 10  2 12 10 10  5 12 10  6  1 10  7 12 10  1 10  2  1 10
   2 12 10  7 10 12  2 12 10  1 10  2  1 10  2 12 10  6 10  5 12  4  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0 10 12 10  1  8 12  2 14  4 14 14  1 10  1  8 10 10 10 10 10  1  8 12
   2 13 10  2 14  5 14 14  1 10 15  2 14  1 12 10  2 14  1 10  1  8 12  2
  12 10  2 14  5  2 14 14  1 10 15  2 14  4 12 10]]
[[10

### One-Hot encoding
Bij dit stukje one-hot encoding ben ik zelf een tijdje vastgelopen, ik snapte wel wat het deed maar niet waarom het nodig is. One-hot encoding is het process van het uitleggen wat de data is in verglijken met elkaar. Een molecuul zou kunnen zijn: "C", "=", "0" : 1, 2, 3 maar als ik deze data direct in een model zou stoppen dan kan het model een verband gaat leggen tussen de hoeveelheid en de afstand van de cijfers, bijvoorbeeld 1 is dichter bij 2 en verder weg van 3. Er wordt dan een verband gelegd tussen de cijfers die eigenlijk alleen maar symbolen voorstellen. One-hot encoding maakt deze nummers een 2d array: 1, 2, 3 wordt dan:


In [14]:
[
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1]
]

[[1, 0, 0], [0, 1, 0], [0, 0, 1]]

In het bovenstaande heb ik de lengte van de vocabulary, in dit geval 3, gepakt en heb op de die het getal van het atoom aan gaf een 1 gezet. Dit zorgt ervoor dat er geen verband meer is tussen de verschillende getallen, alles is een 1. Hieronder staat de code die ik gebruik om dit op mijn moleculen te doen. tensorflow.keras.utils heeft hier een functie voor die dit automatisch doet voor een getal.

In [37]:
from tensorflow.keras.utils import to_categorical

def one_hot_encode_tokens(tokens):
    return to_categorical(tokens, num_classes=len(vocab) + 1)

X_oh = one_hot_encode_tokens(X)
y_oh = one_hot_encode_tokens(y)
print(X_oh.shape)
print(X_oh[0])

(5, 64, 17)
[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


### Model bouwen
Nu is het dan ook eindelijk tijd voor het bouwen van het model. Sequential is de manier waarop het model gebouwd kan worden, in dit geval kun je dus een model layer voor layer opbouwen. LSTM is Long Short-Term Memory, dit is de laag in het RNN die er voor zorgt dat sequential leren mogelijk is. Dense is een layer die ik kan toevoegen die standaard met alle layers ervoor en erna connect, een soort tussenlayer dus. En de laatste import, Timedustributed is een layer die gelijk is aan Dense, deze layer doet ook elk punt van 2 layers met elkaar verbinden. TimeDistrubuted zorgt ervoor dat je een output kan krijgen in elke stap van je process wat nodig is voor het nakijken van gegenereede sequenties.

Bij deze stukjes code heb ik ook de import per code vak gezet, het kan zijn dat dit trainen een tijdje duurt en je dus ook liever niet zit te wachten tot er een klaar is, door de imports per code blok te doen kun je deze dus ook apart runnen.

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed



Tijdens het maken van dit project ben ik op een extra Model uitgekomen GRU (Gated Recurrent Unit), dit is speciaal gemaakt voor het genereren van Sequenties maar is iets simpeler en sneller om mee te leren dan LSTM. Maar ondanks dat het simpeler en sneller is zou het nog steeds ongeveer het zelfde moeten presteren. Ik heb er daarom ook voor gekozen om hier mee te gaan beginnen.

Ik heb dus hieronder het model gebouwd maar wat doet alles nu? Voor het bouwen van het model ben ik begonnen met een GRU layer met 128 Units (neurons) dit heb ik gekozen omdat het over het algemeen aangeraden wordt om met deze hoeveelheid te beginnen. De hoeveelheid van deze units bepaald eigenlijk hoe groot de memory van het model is, meer units betekent meer learning power maar ook meer kans op overfitten. return_sequences zorgt ervoor dat ik ook een sequence terugkrijg in plaats van maar 1 uitkomst. Input_shape is een array omdat ik een molecuul dus meerdere inputs tegelijk wil geven.

In [66]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed, GRU

vocab_size = len(vocab)

model = Sequential([
    GRU(128, return_sequences=True, input_shape=(X_oh.shape[1], vocab_size)),
    TimeDistributed(Dense(vocab_size, activation="softmax")) 
])

Nadat het model is gebouwd moet dit ook compiled worden, het compiled van het model doet eigenlijk vast zetten hoe het model eruit ziet en de manier van leren ingeven. Met de loss function kijk je hoe fout het model is vergleken met de labels die je hebt meegegeven, categorical_crossentropy is speciaal gemaakt voor sequences dus goed voor dit project. Een lage loss is goed en een hoge loss is slecht. De optimizer functie bepaald hoe de weights van het trainen worden veranderd ten opzichte van de loss. Het model gaat dus als volgt trainen, de input is een molecuul en hierop word een gok gedaan doormiddel van de weights en bias. Na deze gok word er door de loss function gekeken hoe ver deze gok naast het getal zit wat het moet zijn. Doormiddel van de optimizer worden de biases in het model aangepast en dit wordt voor elk training molecuul gedaan.

In [68]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru_1 (GRU)                     │ (None, 64, 128)        │        56,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_1              │ (None, 64, 16)         │         2,064 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 58,128 (227.06 KB)

 Trainable params: 58,128 (227.06 KB)

 Non-trainable params: 0 (0.00 B)

Bij het compilen van het model is mij opgevallen dat deze fysiek word opgeslagen, hieronder een functie om deze cache te clearen.

In [3]:
import tensorflow as tf
tf.keras.backend.clear_session()

ModuleNotFoundError: No module named 'tensorflow'

Nu het trainen van het model